# dHybridR M06_th45 2D Analysis
#### Here we analyze a specific simulation, assuming that FPC data has already been generated

Note: we generate most plots twice, once to make appear in notebook, once to save as png. TODO: streamline this

In [1]:
#Need this to add libraries in parent folder
import sys
sys.path.append('..')
sys.path.append('.')
sys.path.append('../lib')

In [2]:
import lib.analysis as anl 
import lib.array_ops as ao
import lib.data_h5 as dh5
import lib.data_netcdf4 as dnc
import lib.fpc as fpc
import lib.frametransform as ft
import lib.metadata as md

import lib.plot.oned as plt1d
import lib.plot.twod as plt2d
import lib.plot.debug as pltdebug
import lib.plot.fourier as pltfr
import lib.plot.resultsmanager as rsltmng
import lib.plot.velspace as pltvv

import os
import math
import numpy as np

/home/collbrown/dHybridR/analysis/dHybridRAnalysis/notebooks/../lib/data_h5.py:148: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if path[-1] is not '/': path = path + '/'
/home/collbrown/dHybridR/analysis/dHybridRAnalysis/notebooks/../lib/data_h5.py:256: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if path[-1] is not '/': path = path + '/'
/home/collbrown/dHybridR/analysis/dHybridRAnalysis/notebooks/../lib/data_h5.py:318: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if path[-1] is not '/': path = path + '/'


## Load data

In [3]:
# load analysis file
analysisflnm = '../analysisinputsperp/m06_th45input2d.txt'
path,resultsdir,vmax,dv,numframe,dx,xlim,ylim,zlim = anl.analysis_input(flnm = analysisflnm)
path_particles = path+'Output/Raw/Sp01/raw_sp01_{:08d}.h5'

In [4]:
#make results directory
resultstag = 'M06_th452d'
resultsdir = resultstag+'results/'
os.system('mkdir ' + resultsdir)

256

In [5]:
#load fields
dfields = dh5.field_loader(path=path,num=numframe,is2d3v=True)

#load flow data
dflow = dh5.flow_loader(path=path,num=numframe,is2d3v=True)


In [6]:
dfields.keys()

dict_keys(['ex', 'ex_xx', 'ex_yy', 'ey', 'ey_xx', 'ey_yy', 'ez', 'ez_xx', 'ez_yy', 'bx', 'bx_xx', 'bx_yy', 'by', 'by_xx', 'by_yy', 'bz', 'bz_xx', 'bz_yy', 'Vframe_relative_to_sim'])

In [7]:
dparticles = dh5.read_particles(path_particles,numframe,is2d3v=True)

In [8]:
dparticles

{'p1': array([-6.086775 , -5.8394556, -6.4944506, ..., -5.303186 , -4.76487  ,
        -6.240882 ], dtype=float32),
 'p2': array([-1.7467011 , -1.5586746 , -0.62981725, ..., -1.058637  ,
        -1.2323486 , -1.5020976 ], dtype=float32),
 'p3': array([ 0.09673929,  0.31545776, -0.49248534, ...,  0.6542207 ,
        -0.4666493 ,  0.34082678], dtype=float32),
 'x1': array([34.21381 , 34.255028, 34.15678 , ..., 71.05757 , 70.42394 ,
        70.0771  ], dtype=float32),
 'x2': array([ 2.5818243,  2.7723327,  2.909247 , ..., 47.98753  , 47.995106 ,
        47.975784 ], dtype=float32),
 'Vframe_relative_to_sim': 0.0}

In [9]:
all_dfields = dh5.all_dfield_loader(path=path, verbose=False,is2d3v=True)

In [11]:
dfields['ex'].shape

(192, 384)

### Lorentz transform fields and shift frame of flow

In [10]:
vshock, xshockvals = ft.shock_from_ex_cross(all_dfields)

IndexError: invalid index to scalar variable.

In [ ]:
## (Optional) Check if shock was tracked well. We use the back half in time to fit to a line
#plt1d.time_stack_line_plot(all_dfields,'ex',pts = xshockvals)

In [ ]:
#loretnz transform fields
dfields = ft.lorentz_transform_vx(dfields,vshock)
_fields = []
for k in range(0,len(all_dfields['dfields'])):
    _fields.append(ft.lorentz_transform_vx(all_dfields['dfields'][k],vshock))
all_dfields['dfields'] = _fields

#tranform flow
dflow = ft.transform_flow(dflow,vshock)

In [ ]:
# (Optional) plot all the lorentz transformed fields
plt1d.plot_all_fields(dfields, axis='_xx', xxindex = 0, yyindex = 0, zzindex = 0, flnm=resultsdir+resultstag+'fields.png')
plt1d.plot_all_fields(dfields, axis='_xx', xxindex = 0, yyindex = 0, zzindex = 0)

In [ ]:
# (Optional) plot flow in the etimated frame of the shock
plt1d.plot_all_flow(dflow, axis='_xx', xxindex = 0, yyindex = 0, zzindex = 0, flnm=resultsdir+resultstag+'flow.png')
plt1d.plot_all_flow(dflow, axis='_xx', xxindex = 0, yyindex = 0, zzindex = 0)

### Load FPC data

In [ ]:
ncflnm = '../M06_th45results/m06_th45.nc'
Hist, CEx, CEy, CEz, vx, vy, vz, x, enerCEx, enerCEy, enerCEz, Vframe_relative_to_sim, metadata_in, params_in = dnc.load3Vnetcdf4(ncflnm)

### Look for signature bounds

In [ ]:
startval = 19.0#lower bound for sda metadata = 1 (di)
endval = 25.5#upper bound for sda metadata = 1 (di)

In [ ]:
#option one, use Ex(x,y=0,z=0) to find SDA bounds. Should range from the start of the ramp to the first zero crossing
yyindex = 0
zzindex = 0
plt1d.plot_field(dfields, 'ex', axis='_xx', yyindex = yyindex, zzindex = zzindex, axvx1 = startval, axvx2 = endval,flnm=resultsdir+resultstag+'shockbounds.png')
plt1d.plot_field(dfields, 'ex', axis='_xx', yyindex = yyindex, zzindex = zzindex, axvx1 = startval, axvx2 = endval)

In [ ]:
#option two (better but harder), look at the FPC signatures and see when the shock starts and stops

#Plot FPCs and dist funcs at start val-------------------------------------------------------------
#grab index of starval
i = ao.find_nearest(x, startval)

#Project onto 2d axis
H_xy = ao.array_3d_to_2d(Hist[i],'xy')
H_xz = ao.array_3d_to_2d(Hist[i],'xz')
H_yz = ao.array_3d_to_2d(Hist[i],'yz')
CEx_xy = ao.array_3d_to_2d(CEx[i],'xy')
CEx_xz = ao.array_3d_to_2d(CEx[i],'xz')
CEx_yz = ao.array_3d_to_2d(CEx[i],'yz')
CEy_xy = ao.array_3d_to_2d(CEy[i],'xy')
CEy_xz = ao.array_3d_to_2d(CEy[i],'xz')
CEy_yz = ao.array_3d_to_2d(CEy[i],'yz')
CEz_xy = ao.array_3d_to_2d(CEz[i],'xy')
CEz_xz = ao.array_3d_to_2d(CEz[i],'xz')
CEz_yz = ao.array_3d_to_2d(CEz[i],'yz')

#plot
pltvv.plot_cor_and_dist_supergrid(vx, vy, vz, vmax,
                                H_xy, H_xz, H_yz,
                                CEx_xy,CEx_xz, CEx_yz,
                                CEy_xy,CEy_xz, CEy_yz,
                                CEz_xy,CEz_xz, CEz_yz,flnm=resultsdir+resultstag+'superplotstartval.png')
pltvv.plot_cor_and_dist_supergrid(vx, vy, vz, vmax,
                                H_xy, H_xz, H_yz,
                                CEx_xy,CEx_xz, CEx_yz,
                                CEy_xy,CEy_xz, CEy_yz,
                                CEz_xy,CEz_xz, CEz_yz)

In [ ]:
#plot where above signature is
plt1d.plot_field(dfields, 'ex', axis='_xx', yyindex = yyindex, zzindex = zzindex, axvx1 = startval,flnm=resultsdir+resultstag+'startvalpos.png')
plt1d.plot_field(dfields, 'ex', axis='_xx', yyindex = yyindex, zzindex = zzindex, axvx1 = startval)

In [ ]:
#Plot FPCs and dist funcs at end val--------------------------------------------------------------
#grab index of starval
i = ao.find_nearest(x, endval)

#Project onto 2d axis
H_xy = ao.array_3d_to_2d(Hist[i],'xy')
H_xz = ao.array_3d_to_2d(Hist[i],'xz')
H_yz = ao.array_3d_to_2d(Hist[i],'yz')
CEx_xy = ao.array_3d_to_2d(CEx[i],'xy')
CEx_xz = ao.array_3d_to_2d(CEx[i],'xz')
CEx_yz = ao.array_3d_to_2d(CEx[i],'yz')
CEy_xy = ao.array_3d_to_2d(CEy[i],'xy')
CEy_xz = ao.array_3d_to_2d(CEy[i],'xz')
CEy_yz = ao.array_3d_to_2d(CEy[i],'yz')
CEz_xy = ao.array_3d_to_2d(CEz[i],'xy')
CEz_xz = ao.array_3d_to_2d(CEz[i],'xz')
CEz_yz = ao.array_3d_to_2d(CEz[i],'yz')

#plot
pltvv.plot_cor_and_dist_supergrid(vx, vy, vz, vmax,
                                H_xy, H_xz, H_yz,
                                CEx_xy,CEx_xz, CEx_yz,
                                CEy_xy,CEy_xz, CEy_yz,
                                CEz_xy,CEz_xz, CEz_yz,flnm=resultsdir+resultstag+'superplotendval.png')
pltvv.plot_cor_and_dist_supergrid(vx, vy, vz, vmax,
                                H_xy, H_xz, H_yz,
                                CEx_xy,CEx_xz, CEx_yz,
                                CEy_xy,CEy_xz, CEy_yz,
                                CEz_xy,CEz_xz, CEz_yz)

In [ ]:
#plot where above signature is
plt1d.plot_field(dfields, 'ex', axis='_xx', yyindex = yyindex, zzindex = zzindex, axvx1 = endval,flnm=resultsdir+resultstag+'endvalpos.png')
plt1d.plot_field(dfields, 'ex', axis='_xx', yyindex = yyindex, zzindex = zzindex, axvx1 = endval)

In [ ]:
#set metadata using add Metadata script
cmd = 'python3 ../scripts/addMetadata.py ' + analysisflnm + ' ' + ncflnm + ' ' + str(startval) + ' ' + str(endval)
os.system(cmd)

### Look for shock rippling

#### Make pmesh of fields

In [ ]:
#plot xy plane at z=0 
plt2d.make_field_pmesh(dfields,'bz','xy',takeaxisaverage=False, xxindex=float('nan'), yyindex=float('nan'), zzindex=0, xlimmin=None,xlimmax=None,flnm=resultsdir+resultstag+'Bzpmesh.png')
plt2d.make_field_pmesh(dfields,'bz','xy',takeaxisaverage=False, xxindex=float('nan'), yyindex=float('nan'), zzindex=0, xlimmin=None,xlimmax=None)

In [ ]:
#zoom in on potential ripple xy plane at z=0
plt2d.make_field_pmesh(dfields,'bz','xy',takeaxisaverage=False, xxindex=float('nan'), yyindex=float('nan'), zzindex=0, xlimmin=startval,xlimmax=endval,flnm=resultsdir+resultstag+'Bzpeshzoomed.png')
plt2d.make_field_pmesh(dfields,'bz','xy',flnm = '',takeaxisaverage=False, xxindex=float('nan'), yyindex=float('nan'), zzindex=0, xlimmin=startval,xlimmax=endval)

#### Make stacked line plot

In [ ]:
plt1d.plot_stack_field_along_x(dfields,'bz','_yy',yyindex=0,zzindex=0,xlow=startval,xhigh=endval,flnm=resultsdir+resultstag+'stackedripple.png')
plt1d.plot_stack_field_along_x(dfields,'bz','_yy',yyindex=0,zzindex=0,xlow=startval,xhigh=endval)

### Diagnose corrugation instability
#### look at number of significant wave modes in fourier space
#### still WIP

In [ ]:
shockval = 15.0 #di, some slice in the ramp of the shock
i = ao.find_nearest(x, shockval)
pltfr.plot_fft_norm(dfields,'bz','yz',takeaxisaverage=False, xxindex=i, yyindex=float('nan'), zzindex=float('nan'), plotlog = False, xaxislim=None, yaxislim=None,flnm=resultsdir+resultstag+'2dfftinramp.png')
pltfr.plot_fft_norm(dfields,'bz','yz',flnm = '',takeaxisaverage=False, xxindex=i, yyindex=float('nan'), zzindex=float('nan'), plotlog = False, xaxislim=None, yaxislim=None)

### Generate sweep gifs

#### Generate sweeps of superplot

In [ ]:
pltvv.make_superplot_gif(vx,vy,vz,vmax,Hist,CEx,CEy,CEz,x,resultsdir+'superplotscanfiles',resultsdir+resultstag+'superplot.gif')

In [ ]:
rsltmng.make_gif_from_folder(resultsdir+'superplotscanfiles',resultsdir+resultstag+'superplot.gif')

In [ ]:
plt1d.make_field_scan_gif(dfields,'ex',resultsdir+'fieldscanfiles',resultsdir+resultstag+'scan.gif')

In [ ]:
rsltmng.make_gif_from_folder(resultsdir+'fieldscanfiles',resultsdir+resultstag+'fieldscan.gif')